In [2]:
#!/usr/bin/python
path = "D:/Users/hjcam/Documents/Python Scripts/ECTE355 Project/"
os.chdir( path )

In [3]:
data = {"Normal":{},"Aggressive":{},"Drowsy":{}}

for behaviour in os.listdir('Dataset'):
    print(behaviour)
    for drive in os.listdir('Dataset/'+behaviour):
        dataset = []
        acc = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/RAW_ACCELEROMETERS.txt')
        gps = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/RAW_GPS.txt')
        lane = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/PROC_LANE_DETECTION.txt')
        veh = np.loadtxt('Dataset/' + behaviour + '/' + drive + '/PROC_VEHICLE_DETECTION.txt')
        
        acc_reduced = acc[:,[0,5,6,7,8,9,10]] #time, X KF,Y KF, Z KF, Roll, Pitch, Yaw
        gps_reduced = gps[:,[0,1]] #time, speed
        lane_reduced = lane[:,[0,1]] #time, lane center
        veh_reduced = veh[:,[0,1,2]] #time, dist. to veh, time to vehicle
        
        acc_data = pd.DataFrame(acc_reduced, columns = ['Time','X KF','Y KF', 'Z KF', 'Roll', 'Pitch', 'Yaw'])
        acc_data.set_index('Time',inplace = True)
        gps_data = pd.DataFrame(gps_reduced, columns = ['Time','Speed'])
        gps_data.set_index('Time',inplace = True)
        lane_data = pd.DataFrame(lane_reduced, columns = ['Time','Lane_X'])
        lane_data.set_index('Time',inplace = True)
        veh_data = pd.DataFrame(veh_reduced, columns = ['Time','Veh_Dist','Veh_Time'])
        veh_data.set_index('Time',inplace = True)
        
        acc_data = acc_data.groupby(acc_data.index).first()
        gps_data = gps_data.groupby(gps_data.index).first()
        lane_data = lane_data.groupby(lane_data.index).first()
        veh_data = veh_data.groupby(veh_data.index).first()
        
        combined_data = pd.concat([acc_data, gps_data], axis=1, sort=False)
        
        combined_data['Speed'].interpolate(method = 'index',inplace = True)
        #combined_data['Lane_X'].interpolate(method = 'index',inplace = True)
        #combined_data['Veh_Dist'] = combined_data['Veh_Dist'].fillna(method='ffill')
        #combined_data['Veh_Time'] = combined_data['Veh_Time'].fillna(method='ffill')
        combined_data.dropna(inplace = True)
        combined_data = combined_data[['X KF','Y KF', 'Z KF', 'Roll', 'Pitch', 'Yaw','Speed']]#],'Lane_X','Veh_Dist','Veh_Time']]
        
        data[behaviour].update({drive:combined_data})

Aggressive
Drowsy
Normal


# Creating the overall formatted dataset

In [4]:
def split_sequences(sequences, n_steps, behaviour, n_overlap):
    X , y = list() , list()
    #for i in range(len(sequences)):
    for i in np.arange(0,len(sequences),n_overlap,dtype = np.int16):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        #X.append(list(map(list, zip(*seq_x))))
        X.append(seq_x)
        if behaviour == 'Normal':
            y.append([0])
        elif behaviour == 'Drowsy':
            y.append([1])
        elif behaviour == 'Aggressive':
            y.append([2])
            
    return X , y